# EERIE Data at DKRZ

This notebook guides EERIE data users and explains how to find and load data available at DKRZ.

The notebook works well within the `python3/unstable` kernel.

In [1]:
from copy import deepcopy
def find_data_sources(catalog,name=None):
    newname='.'.join(
        [ a 
         for a in [name, catalog.name]
         if a
        ]
    )
    data_sources = []

    for key, entry in catalog.items():
        if isinstance(entry, intake.catalog.Catalog):
            if newname == "main":
                newname = None
            # If the entry is a subcatalog, recursively search it
            data_sources.extend(find_data_sources(entry, newname))
        elif isinstance(entry, intake.source.base.DataSource):
            if newname:
                data_sources.append(newname+"."+key)
            else:
                data_sources.append(key)

    return data_sources

All data relevant for the project is referenced in the main DKRZ-EERIE Catalog:

In [2]:
import intake
eerie_cat=intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/dkrz/disk/model-output/main.yaml")
eerie_cat

main:
  args:
    path: https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/dkrz/disk/model-output/main.yaml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


In [3]:
sources=find_data_sources(eerie_cat)

In [4]:
sources=[a for a in sources if not "json" in a and not "csv" in a ]

In [5]:
hadgem=[a for a in sources if "hadgem" in a]

In [6]:
def test_entry(cat,a):
    assert cat[a]

def test_describe(cat,a):
    assert cat[a].describe()

def test_open(cat,a):
    assert cat[a].to_dask()

def test_load(cat,a):
    ds = cat[a].to_dask()
    if "time" in ds.dims:
        droplist=[a for a in ds.variables if len(ds[a].dims) == 0]
        if droplist:
            ds=ds.drop(droplist)
        assert ds.isel(time=0).load()
    else:
        assert ds.load()

In [8]:
from tqdm import tqdm
failed=[]
for a in tqdm(sources):
    print(a)
    if "grid" in a:
        continue
    for func in [test_entry, test_describe, test_open,test_load]:
        try:
            func(eerie_cat,a)
        except:
            failed.append(a)
            print(func.__name__+" fails for "+a)    

  0%|          | 0/113 [00:00<?, ?it/s]

icon-esm-er.eerie-control-1950.atmos.gr025.2d_daily_max


  1%|          | 1/113 [00:00<00:43,  2.55it/s]

icon-esm-er.eerie-control-1950.atmos.gr025.2d_daily_mean


  2%|▏         | 2/113 [00:02<02:06,  1.14s/it]

...

 98%|█████████▊| 111/113 [07:07<00:01,  1.35it/s]

hadgem3-gc5-n216-orca025.eerie-picontrol.ocean.gr025.daily


/sw/spack-levante/mambaforge-23.1.0-1-Linux-x86_64-3boc6i/lib/python3.10/site-packages/xarray/conventions.py:427: SerializationWarning: variable 'siconc' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/sw/spack-levante/mambaforge-23.1.0-1-Linux-x86_64-3boc6i/lib/python3.10/site-packages/xarray/conventions.py:427: SerializationWarning: variable 'sithick' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/sw/spack-levante/mambaforge-23.1.0-1-Linux-x86_64-3boc6i/lib/python3.10/site-packages/xarray/conventions.py:427: SerializationWarning: variable 'siu' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/sw/spack-levante/mambaforge-23.1.0-1-Linux-x86_64-3boc6i/lib/python3.10/site-packages/xarray/conventions.py:427: SerializationWarning: variable 'siv' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new

hadgem3-gc5-n216-orca025.eerie-picontrol.ocean.gr025.monthly


100%|██████████| 113/113 [07:09<00:00,  3.80s/it]
